In [25]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import re
import io
import time

# Load environment variables
load_dotenv('../../.env')

# Create the engine
def get_engine():
    db_user = os.getenv('DB_USER')
    db_password = os.getenv('DB_PASSWORD') 
    db_host = os.getenv('DB_HOST')
    db_port = os.getenv('DB_PORT')
    db_database = os.getenv('DB_DATABASE')
    
    connection_string = f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}"
    
    engine = create_engine(
        connection_string,
        connect_args={
            "sslmode": "require"  # Azure PostgreSQL requires SSL
        }
    )
    return engine

def format_query(query_text, params):
    """
    Format query by replacing $(param_name) with actual values
    Similar to the Node.js formatQuery function
    """
    formatted_query = query_text
    for key, value in params.items():
        # Replace $(key) with the actual value
        pattern = f"\\$\\({key}\\)"
        if isinstance(value, str):
            formatted_query = re.sub(pattern, f"'{value}'", formatted_query)
        else:
            formatted_query = re.sub(pattern, str(value), formatted_query)
    
    return formatted_query

def run_query(query_filename, rawQuery=None, params=None, engine=None):
    """
    Run a SQL query from a file with parameters
    Similar to the Node.js runQuery function
    """
    if engine is None:
        engine = get_engine()
    
    if params is None:
        params = {}
    
    try:
        if rawQuery is None:
            # Read the SQL file
            query_path = os.path.join('queries', query_filename)
            with open(query_path, 'r') as file:
                query_text = file.read()
        else:
            query_text = rawQuery
        
        # Format the query with parameters
        formatted_query = format_query(query_text, params)
        
        # Execute the query and return DataFrame
        df = pd.read_sql(formatted_query, engine)
        return df
        
    except Exception as error:
        print(f"Error while running query: {query_filename} -> {error}")
        print(f"Formatted query:\n{formatted_query}")
        raise error
    

def write_to_table(df, schema_name, table_name, engine=None):
    """
    Write a DataFrame to a table
    """
    if engine is None:
        engine = get_engine()
     # 1) Create empty table matching df’s schema
    df.head(0).to_sql(
        table_name,
        engine,
        schema=schema_name,
        if_exists='replace',
        index=False
    )

    # 2) stream via COPY with quoted columns
    buffer = io.StringIO()
    df.to_csv(buffer, index=False, header=False)
    buffer.seek(0)

    cols = ', '.join(f'"{col}"' for col in df.columns)
    copy_sql = (
        f'COPY "{schema_name}"."{table_name}" ({cols}) '
        'FROM STDIN WITH (FORMAT csv)'
    )

    conn   = engine.raw_connection()
    cursor = conn.cursor()
    try:
        cursor.copy_expert(copy_sql, buffer)
        conn.commit()
    finally:
        cursor.close()
        conn.close()



In [44]:
import pandas as pd 

# based on the pay period and merchant id, get the pricing info
def get_period_merchant_pricing_info(price_period, merchant_id, use_scraping=True):
    """
    Get pricing info using SQL file with parameters
    """
    params = {
        'price_period': price_period,
        'merchant_id': merchant_id
    }
    if use_scraping:
        df = run_query('get_pricing_info_with_scraping_price.sql', params)
    else:
        df = run_query('get_pricing_info_with_sales.sql', params)
    return df


# filter for item_id = 8225801
df = get_period_merchant_pricing_info(10, 17)
d_8225801 = df[df['item_id'] == 8225801]
d_8225801.head(20)


,bb_id,store_id,item_id,name,day,regular_price,sale_price,selling_price,unit_cost,units_sold
0,2319.0,1173,8225801,Golden Curry Sauce Mix Mild 7.8oz(220g),2024-11-21 00:00:00+00:00,4.99,3.99,3.99,2.5306,2.0
1,2319.0,1173,8225801,Golden Curry Sauce Mix Mild 7.8oz(220g),2024-11-21 00:00:00+00:00,4.99,3.99,4.99,2.5306,4.0


In [17]:
# all pricing info for all pay periods and merchants
def get_all_pricing_info(use_scraping=True):
    """
    Get all pricing info using SQL file with parameters
    """
    params = {
    }
    if use_scraping:
        df = run_query('get_all_pricing_info_with_scraping.sql', params)
    else:
        df = run_query('get_all_pricing_info_with_sales.sql', params)
    return df

df = get_all_pricing_info()
df.head(5)

Error while running query: get_all_pricing_info_with_scraping.sql -> Not an executable object: {}
Formatted query:
{}


ObjectNotExecutableError: Not an executable object: {}

In [46]:
df_all_8225801_10_17 = df[(df['item_id'] == 8225801) & (df['period_id'] == 10) & (df['merchant_id'] == 17)]
df_all_8225801_10_17.head(5)

,bb_id,merchant_id,period_id,store_id,item_id,name,day,regular_price,sale_price,selling_price,unit_cost,units_sold,avg_basket_size
131,2319.0,17,10,1173,8225801,Golden Curry Sauce Mix Mild 7.8oz(220g),2024-11-21 00:00:00+00:00,4.99,3.99,3.99,2.5306,2.0,99.91
132,2319.0,17,10,1173,8225801,Golden Curry Sauce Mix Mild 7.8oz(220g),2024-11-21 00:00:00+00:00,4.99,3.99,4.99,2.5306,4.0,126.58


# for single merchant and period:


In [23]:


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

def get_available_items(df):
    """
    Get all available item IDs and their names from the dataframe
    """
    items = df[['item_id', 'name']].drop_duplicates().sort_values('item_id')
    return items

def calculate_profits(df, item_id):
    """
    Calculate actual profits using unit_cost data
    """
    d = df[df['item_id'] == item_id].copy()
    if d.empty:
        return pd.DataFrame()
    
    # Convert columns to numeric
    numeric_cols = ['regular_price', 'sale_price', 'selling_price', 'units_sold', 'unit_cost']
    for col in numeric_cols:
        if col in d.columns:
            d[col] = pd.to_numeric(d[col], errors='coerce')
    
    # Calculate actual profit using unit_cost
    unit_cost = d['unit_cost'].fillna(0)
    selling_price = d['selling_price'].fillna(0)
    units_sold = d['units_sold'].fillna(0)
    
    d['profit_per_unit'] = selling_price - unit_cost
    d['total_profit'] = d['profit_per_unit'] * units_sold
    d['profit_margin'] = np.where(selling_price > 0, 
                                  (d['profit_per_unit'] / selling_price) * 100, 0)
    
    # Additional useful metrics
    d['revenue'] = selling_price * units_sold
    d['total_cost'] = unit_cost * units_sold
    d['markup_percentage'] = np.where(unit_cost > 0,
                                     (d['profit_per_unit'] / unit_cost) * 100, 0)
    
    return d

def create_item_selector(df):
    """
    Create an interactive widget for selecting and visualizing items
    """
    # Get available items
    available_items = get_available_items(df)
    
    # Create dropdown options (item_id: name format)
    dropdown_options = [(f"{row['item_id']}: {row['name']}", row['item_id']) 
                       for _, row in available_items.iterrows()]
    
    # Create widgets
    item_dropdown = widgets.Dropdown(
        options=dropdown_options,
        description='Select Item:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='600px')
    )
    
    # Buttons
    plot_prices_btn = widgets.Button(description="Plot Prices & Volume", button_style='primary')
    plot_profits_btn = widgets.Button(description="Plot Profit Analysis", button_style='success')
    show_stats_btn = widgets.Button(description="Show All Statistics", button_style='info')
    
    # Output area
    output = widgets.Output()
    
    def plot_prices(b):
        with output:
            clear_output(wait=True)
            if item_dropdown.value:
                plot_price_variance(df, item_dropdown.value)
    
    def plot_profit_analysis(b):
        with output:
            clear_output(wait=True)
            if item_dropdown.value:
                plot_profit_variance(df, item_dropdown.value)
    
    def show_item_statistics(b):
        with output:
            clear_output(wait=True)
            if item_dropdown.value:
                show_price_statistics(df, [item_dropdown.value])
    
    # Bind button clicks
    plot_prices_btn.on_click(plot_prices)
    plot_profits_btn.on_click(plot_profit_analysis)
    show_stats_btn.on_click(show_item_statistics)
    
    # Layout
    ui = widgets.VBox([
        widgets.HTML("<h3>Item Price & Profit Analysis Dashboard</h3>"),
        widgets.HTML(f"<p>Total items available: {len(available_items)}</p>"),
        item_dropdown,
        widgets.HBox([plot_prices_btn, plot_profits_btn, show_stats_btn]),
        output
    ])
    
    return ui

def plot_price_variance(df, item_id):
    """
    Enhanced single item plotting function for prices and volume
    """
    # Filter to that item and sort by date
    d = df[df['item_id'] == item_id].sort_values('day').copy()
    if d.empty:
        print(f"No data for item_id = {item_id}")
        return
    
    # Convert price columns to numeric
    price_cols = ['regular_price', 'sale_price', 'selling_price', 'unit_cost']
    for col in price_cols:
        if col in d.columns:
            d[col] = pd.to_numeric(d[col], errors='coerce')
    
    # Get all price values to determine optimal y-axis range
    all_prices = []
    for col in price_cols:
        if col in d.columns:
            prices = d[col].dropna()
            if not prices.empty:
                all_prices.extend(prices.tolist())
    
    if not all_prices:
        print(f"No price data available for item_id = {item_id}")
        return
    
    min_price = min(all_prices)
    max_price = max(all_prices)
    price_range = max_price - min_price
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
    
    # Top plot: Prices including cost
    if 'regular_price' in d.columns:
        regular_data = d[d['regular_price'].notna()]
        if not regular_data.empty:
            ax1.plot(regular_data['day'], regular_data['regular_price'], 
                    label='Regular Price', marker='o', linewidth=2, color='blue')
    
    if 'sale_price' in d.columns:
        sale_data = d[d['sale_price'].notna()]
        if not sale_data.empty:
            ax1.plot(sale_data['day'], sale_data['sale_price'], 
                    label='Sale Price', marker='s', linewidth=2, color='green')
    
    if 'selling_price' in d.columns:
        selling_data = d[d['selling_price'].notna()]
        if not selling_data.empty:
            ax1.plot(selling_data['day'], selling_data['selling_price'], 
                    label='Selling Price', marker='^', linewidth=2, color='red')
    
    # Add unit cost line
    if 'unit_cost' in d.columns:
        cost_data = d[d['unit_cost'].notna()]
        if not cost_data.empty:
            ax1.plot(cost_data['day'], cost_data['unit_cost'], 
                    label='Unit Cost', marker='x', linewidth=2, color='orange', linestyle='--')
    
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Price ($)')
    ax1.set_title(f'Price Variance for Item {item_id}: {d["name"].iloc[0]}')
    ax1.legend()
    ax1.tick_params(axis='x', rotation=45)
    ax1.grid(True, alpha=0.3)
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:.2f}'))
    
    # Add padding to y-axis
    padding = price_range * 0.05 if price_range > 0 else 0.1
    ax1.set_ylim(max(0, min_price - padding), max_price + padding)
    
    # Bottom plot: Units sold
    if 'units_sold' in d.columns:
        volume_data = d[d['units_sold'].notna()]
        if not volume_data.empty:
            ax2.bar(volume_data['day'], pd.to_numeric(volume_data['units_sold'], errors='coerce'), 
                   alpha=0.7, color='purple', label='Units Sold')
            ax2.set_xlabel('Date')
            ax2.set_ylabel('Units Sold')
            ax2.set_title('Sales Volume')
            ax2.tick_params(axis='x', rotation=45)
            ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def plot_profit_variance(df, item_id):
    """
    Plot actual profit analysis using unit_cost data
    """
    # Calculate profits
    profit_data = calculate_profits(df, item_id)
    if profit_data.empty:
        print(f"No data for item_id = {item_id}")
        return
    
    # Filter out rows with no sales
    profit_data = profit_data[profit_data['units_sold'] > 0].sort_values('day')
    
    if profit_data.empty:
        print(f"No sales data for item_id = {item_id}")
        return
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # Plot 1: Profit per unit over time
    ax1.plot(profit_data['day'], profit_data['profit_per_unit'], 
             marker='o', linewidth=2, color='green', label='Profit per Unit')
    ax1.axhline(y=0, color='red', linestyle='--', alpha=0.7, label='Break-even')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Profit per Unit ($)')
    ax1.set_title('Actual Profit per Unit Over Time')
    ax1.tick_params(axis='x', rotation=45)
    ax1.grid(True, alpha=0.3)
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:.2f}'))
    ax1.legend()
    
    # Plot 2: Total profit vs revenue vs cost
    width = 0.25
    days = profit_data['day']
    x = np.arange(len(days))
    
    ax2.bar(x - width, profit_data['revenue'], width, label='Revenue', alpha=0.8, color='blue')
    ax2.bar(x, profit_data['total_cost'], width, label='Total Cost', alpha=0.8, color='red')
    ax2.bar(x + width, profit_data['total_profit'], width, label='Total Profit', alpha=0.8, color='green')
    
    ax2.set_xlabel('Date')
    ax2.set_ylabel('Amount ($)')
    ax2.set_title('Revenue vs Cost vs Profit by Day')
    ax2.set_xticks(x)
    ax2.set_xticklabels([d.strftime('%m-%d') for d in days], rotation=45)
    ax2.grid(True, alpha=0.3)
    ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:.2f}'))
    ax2.legend()
    
    # Plot 3: Profit margin over time
    ax3.plot(profit_data['day'], profit_data['profit_margin'], 
             marker='s', linewidth=2, color='purple', label='Profit Margin %')
    ax3.axhline(y=0, color='red', linestyle='--', alpha=0.7, label='Break-even')
    ax3.set_xlabel('Date')
    ax3.set_ylabel('Profit Margin (%)')
    ax3.set_title('Profit Margin Over Time')
    ax3.tick_params(axis='x', rotation=45)
    ax3.grid(True, alpha=0.3)
    ax3.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:.1f}%'))
    ax3.legend()
    
    # Plot 4: Cumulative profit
    profit_data['cumulative_profit'] = profit_data['total_profit'].cumsum()
    ax4.plot(profit_data['day'], profit_data['cumulative_profit'], 
             marker='^', linewidth=3, color='navy', label='Cumulative Profit')
    ax4.axhline(y=0, color='red', linestyle='--', alpha=0.7)
    ax4.set_xlabel('Date')
    ax4.set_ylabel('Cumulative Profit ($)')
    ax4.set_title('Cumulative Profit Over Time')
    ax4.tick_params(axis='x', rotation=45)
    ax4.grid(True, alpha=0.3)
    ax4.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:.2f}'))
    ax4.fill_between(profit_data['day'], profit_data['cumulative_profit'], 
                     alpha=0.3, color='navy')
    
    plt.suptitle(f'Actual Profit Analysis for Item {item_id}: {profit_data["name"].iloc[0]}', 
                 fontsize=16, y=0.98)
    plt.tight_layout()
    plt.show()

def show_price_statistics(df, item_ids):
    """
    Show detailed statistics including actual profit analysis
    """
    print("="*80)
    print("COMPREHENSIVE PRICE & ACTUAL PROFIT STATISTICS")
    print("="*80)
    
    for item_id in item_ids:
        profit_data = calculate_profits(df, item_id)
        if profit_data.empty:
            continue
        
        # Filter to sales data only for profit calculations
        sales_data = profit_data[profit_data['units_sold'] > 0]
            
        print(f"\nItem {item_id}: {profit_data['name'].iloc[0]}")
        print("-" * 80)
        
        # Price statistics including cost
        price_cols = ['regular_price', 'sale_price', 'selling_price', 'unit_cost']
        for col in price_cols:
            if col in profit_data.columns and not profit_data[col].isna().all():
                prices = profit_data[col].dropna()
                if not prices.empty:
                    print(f"{col.replace('_', ' ').title():<20}: ${prices.min():.2f} - ${prices.max():.2f} (avg: ${prices.mean():.2f}, std: ${prices.std():.2f})")
        
        # Volume statistics
        if not profit_data['units_sold'].isna().all():
            units = profit_data['units_sold'].dropna()
            if not units.empty:
                print(f"{'Units Sold':<20}: {units.min():.0f} - {units.max():.0f} (avg: {units.mean():.1f}, total: {units.sum():.0f})")
        
        # Actual profit statistics (only for days with sales)
        if not sales_data.empty:
            print(f"\n{'ACTUAL PROFIT ANALYSIS':<20}:")
            print(f"{'Profit per Unit':<20}: ${sales_data['profit_per_unit'].min():.2f} - ${sales_data['profit_per_unit'].max():.2f} (avg: ${sales_data['profit_per_unit'].mean():.2f})")
            print(f"{'Total Revenue':<20}: ${sales_data['revenue'].sum():.2f}")
            print(f"{'Total Cost':<20}: ${sales_data['total_cost'].sum():.2f}")
            print(f"{'Total Profit':<20}: ${sales_data['total_profit'].sum():.2f}")
            print(f"{'Avg Daily Profit':<20}: ${sales_data['total_profit'].mean():.2f}")
            print(f"{'Profit Margin':<20}: {sales_data['profit_margin'].min():.1f}% - {sales_data['profit_margin'].max():.1f}% (avg: {sales_data['profit_margin'].mean():.1f}%)")
            print(f"{'Markup %':<20}: {sales_data['markup_percentage'].min():.1f}% - {sales_data['markup_percentage'].max():.1f}% (avg: {sales_data['markup_percentage'].mean():.1f}%)")
            
            # Profitability metrics
            profitable_days = len(sales_data[sales_data['total_profit'] > 0])
            total_sales_days = len(sales_data)
            profitability_rate = (profitable_days / total_sales_days * 100) if total_sales_days > 0 else 0
            print(f"{'Profitable Days':<20}: {profitable_days}/{total_sales_days} ({profitability_rate:.1f}%)")
            
            # ROI calculation
            if sales_data['total_cost'].sum() > 0:
                roi = (sales_data['total_profit'].sum() / sales_data['total_cost'].sum()) * 100
                print(f"{'Return on Investment':<20}: {roi:.1f}%")
        
        # Store information
        stores = profit_data['store_id'].unique()
        print(f"\n{'Store Coverage':<20}: {len(stores)} stores ({', '.join(map(str, stores))})")
        
        # Date range
        if not profit_data['day'].empty:
            print(f"{'Date Range':<20}: {profit_data['day'].min().strftime('%Y-%m-%d')} to {profit_data['day'].max().strftime('%Y-%m-%d')}")

# Quick function to show all available items
def show_available_items(df):
    """
    Display all available items in a nice format
    """
    items = get_available_items(df)
    print(f"Available Items ({len(items)} total):")
    print("="*80)
    for _, row in items.iterrows():
        print(f"ID: {row['item_id']:<10} Name: {row['name']}")

# Usage:
print("Setting up interactive dashboard...")
dashboard = create_item_selector(df)
display(dashboard)

# Alternative: Show available items first
print("\n" + "="*80)
show_available_items(df)

Setting up interactive dashboard...



Available Items (3837 total):
ID: 8225800    Name: Frozen Udon 2.75lb(1.25kg)
ID: 8225801    Name: Golden Curry Sauce Mix Mild 7.8oz(220g)
ID: 8225802    Name: Duck Packaged Fz Smoked/Sliced Half Duck (1 Pack)
ID: 8225803    Name: Glass Noodles 15.87oz(450g)
ID: 8225804    Name: Paik Ramyun 4.1 oz(115g) x 4 Packs
ID: 8225805    Name: Paik Jjajang Ramyun 4.94 oz(140g) x 4 Packs
ID: 8225806    Name: Jeongseon Pancake 1.32lb(600g)
ID: 8225807    Name: Marine Choco Boy 20PK 19.75oz(560g)
ID: 8225808    Name: Cooked Black Rice Bowl 7.4oz(210g) 3 Packs
ID: 8225809    Name: Stir-Fried Gan Jjajang Black Bean Sauce 7.48oz(212g)
ID: 8225810    Name: Instant Miso Soup Spinach 5.76oz(172.8g)
ID: 8225811    Name: Neoguri Spicy 4.2oz(120g) 4 Packs
ID: 8225812    Name: Dukboki Sauce 1.76oz(50g)
ID: 8225813    Name: Black Bean Paste with Smoky Flavor 10.58oz(300g)
ID: 8225814    Name: Golden Curry Sauce Mix Medium Hot 7.8oz(220g)
ID: 8225816    Name: Seasoned Green Laver(GIM) Box 10pk 7.1oz (201g)
ID

# for all merchants and all periods
## right now only merchant is hmart because there is no other sale / cost data anywhere

In [28]:
# run the actual query get_all_pricing_info_with_scraping, and store the results in temporary.raw
df = run_query('get_all_pricing_info_with_scraping.sql')
df = df[['bb_id', 'period_id', 'merchant_id', 'store_id', 'item_id', 'name', 'day', 'timezone', 'regular_price', 'scraping_regular_price', 'promo_price', 'avg_selling_price', 'unit_cost', 'units_sold', 'total_basket_size', 'num_baskets', 'avg_basket_size', 'period_start_local', 'period_end_local']]
df.head(20)
# query = """SELECT * FROM temporary.analysis_cleaned;"""
# df = run_query(None, query)
# df.head(20)


,bb_id,period_id,merchant_id,store_id,item_id,name,day,timezone,regular_price,scraping_regular_price,promo_price,avg_selling_price,unit_cost,units_sold,total_basket_size,num_baskets,avg_basket_size,period_start_local,period_end_local
0,8902.0,0,17,1173,8225800,Frozen Udon 2.75lb(1.25kg),2024-06-21,America/New_York,5.99,None,None,5.390000,NaN,1.0,151.32,1,151.320000,2024-06-21,2024-07-05
1,2319.0,0,17,1173,8225801,Golden Curry Sauce Mix Mild 7.8oz(220g),2024-06-21,America/New_York,4.49,None,None,3.490000,NaN,1.0,213.20,1,213.200000,2024-06-21,2024-07-05
2,2397073.0,0,17,1173,8225803,Glass Noodles 15.87oz(450g),2024-06-21,America/New_York,4.99,None,None,4.990000,NaN,1.0,92.00,1,92.000000,2024-06-21,2024-07-05
3,4140.0,0,17,1173,8225812,Dukboki Sauce 1.76oz(50g),2024-06-21,America/New_York,1.99,None,None,1.116000,NaN,10.0,465.38,3,155.126667,2024-06-21,2024-07-05
4,2320.0,0,17,1173,8225814,Golden Curry Sauce Mix Medium Hot 7.8oz(220g),2024-06-21,America/New_York,4.99,None,None,3.193846,NaN,13.0,1060.47,7,151.495714,2024-06-21,2024-07-05
5,6190.0,0,17,1173,8225815,íì´í¸ê³¨ë ì»¤í¼ë¯¹ì¤ 0.42oz(12g) 100ê°...,2024-06-21,America/New_York,24.99,None,None,24.990000,NaN,2.0,404.04,1,404.040000,2024-06-21,2024-07-05
6,7952.0,0,17,1173,8225816,Seasoned Green Laver(GIM) Box 10pk 7.1oz (201g),2024-06-21,America/New_York,19.99,None,None,19.990000,NaN,1.0,213.20,1,213.200000,2024-06-21,2024-07-05
7,2397101.0,0,17,1173,8225817,Viet Huong Three Crabs Fish Sauce 24 fl.oz(682ml),2024-06-21,America/New_York,6.99,None,None,6.990000,NaN,4.0,245.16,2,122.580000,2024-06-21,2024-07-05
8,4070.0,0,17,1173,8225824,Beef Bone Soup 16.9 fl.oz(500ml) 6 Packs,2024-06-21,America/New_York,12.99,None,None,11.090000,NaN,9.0,952.78,5,190.556000,2024-06-21,2024-07-05
9,6958.0,0,17,1173,8225831,(H-Mart Exclusive) Jjajang Ramen Bundle 20.6 O...,2024-06-21,America/New_York,5.99,None,None,4.650714,NaN,14.0,1694.46,10,169.446000,2024-06-21,2024-07-05


In [24]:
# # some analysis of raw_data 

# df_raw_test = df.copy()
# df_raw_test.head(20)

# # filter for item id 8225846 and period 13, 14, 15
# df_raw_test = df_raw_test[df_raw_test['item_id'] == 8225846]
# df_raw_test = df_raw_test[df_raw_test['period_id'].isin([13, 14, 15])]
# df_raw_test.head(40)

df_specific_test = run_query('debug_weird_pricing.sql')
df_specific_test.head(20)

# filter to periods 13 14 15 
df_specific_test = df_specific_test[df_specific_test['period_id'].isin([13, 14, 15])]
df_specific_test.head(40)


,store_id,item_id,day,timezone,period_id,bb_id,merchant_id,name,period_start_local,period_end_local,promo_price,scraping_regular_price,unit_cost,units_sold,total_basket_size,num_baskets,avg_basket_size,avg_selling_price,regular_price
329,1173,8225846,2024-12-20,America/New_York,13,3437,17,Cooked White Rice Box 7.4oz(210g) 12 Ea,2024-12-19,2025-01-02,None,17.99,NaN,16.0,1767.76,8,220.970000,17.765000,17.99
330,1173,8225846,2024-12-21,America/New_York,13,3437,17,Cooked White Rice Box 7.4oz(210g) 12 Ea,2024-12-19,2025-01-02,None,17.99,NaN,4.0,315.68,2,157.840000,17.990000,17.99
331,1173,8225846,2024-12-22,America/New_York,13,3437,17,Cooked White Rice Box 7.4oz(210g) 12 Ea,2024-12-19,2025-01-02,None,17.99,NaN,10.0,618.98,6,103.163333,16.550000,17.99
332,1173,8225846,2024-12-23,America/New_York,13,3437,17,Cooked White Rice Box 7.4oz(210g) 12 Ea,2024-12-19,2025-01-02,None,17.99,NaN,2.0,304.72,2,152.360000,17.990000,17.99
333,1173,8225846,2024-12-24,America/New_York,13,3437,17,Cooked White Rice Box 7.4oz(210g) 12 Ea,2024-12-19,2025-01-02,None,17.99,NaN,6.0,338.32,2,169.160000,17.990000,17.99
334,1173,8225846,2024-12-25,America/New_York,13,3437,17,Cooked White Rice Box 7.4oz(210g) 12 Ea,2024-12-19,2025-01-02,12.99,None,NaN,18.0,2210.10,10,221.010000,12.990000,17.99
335,1173,8225846,2024-12-26,America/New_York,13,3437,17,Cooked White Rice Box 7.4oz(210g) 12 Ea,2024-12-19,2025-01-02,12.99,None,NaN,24.0,2532.08,18,140.671111,11.798333,17.99
336,1173,8225846,2024-12-27,America/New_York,13,3437,17,Cooked White Rice Box 7.4oz(210g) 12 Ea,2024-12-19,2025-01-02,12.99,None,NaN,52.0,2975.38,22,135.244545,11.840000,17.99
337,1173,8225846,2024-12-28,America/New_York,13,3437,17,Cooked White Rice Box 7.4oz(210g) 12 Ea,2024-12-19,2025-01-02,12.99,None,NaN,28.0,2644.32,16,165.270000,12.154286,17.99
338,1173,8225846,2024-12-29,America/New_York,13,3437,17,Cooked White Rice Box 7.4oz(210g) 12 Ea,2024-12-19,2025-01-02,12.99,None,NaN,52.0,4629.18,36,128.588333,12.040000,17.99


# data cleaning and visualizaiton

## cleaning to put in analysis_staging (just populate rows and backfill non price information)

In [29]:
import pandas as pd
import time


def generate_missing_rows(df):
    # 1) make sure 'day' is datetime (sanity check)
    df = df.copy()
    df['day'] = pd.to_datetime(df['day'])

    # 2) find the first date each item was store in a specific store_id
    sold_mask = df['units_sold'] > 0
    first_sold = (
        df[sold_mask]
        .groupby(['item_id', 'store_id'])['day']
        .min()
        .rename('first_sold')
    )

    # 3) get the overall last date
    last_date = df['day'].max()

    # 4) build the full calendar for each (item_id, store_id)
    records = []
    for (item_id, store_id), start_date in first_sold.items():
        # daily frequency from first sold to last_date
        full_days = pd.date_range(start=start_date, end=last_date, freq='D')
        records.extend(
            {'item_id': item_id, 'store_id': store_id, 'day': d}
            for d in full_days
        )
    full_calendar = pd.DataFrame(records)

    # 5) merge original data onto the full calendar
    df_complete = full_calendar.merge(
        df,
        on=['item_id', 'store_id', 'day'],
        how='left',
        suffixes=('', '_orig')
    )

    # backfilled information handelded in a different method

    return df_complete

def fill_metadata_only(df):
    # 1) sort once so ffill/bfill makes sense
    df = df.sort_values(['item_id','store_id','day']).reset_index(drop=True)

    # 2) pick only the columns you actually want to fill
    fill_cols = ['bb_id','timezone','period_id','merchant_id','name', 'period_start_local', 'period_end_local']
    existing = [c for c in fill_cols if c in df]

    # 3) do a group‐wise ffill→bfill on those columns
    df[existing] = (
        df
        .groupby(['item_id','store_id'])[existing]
        .transform(lambda s: s.ffill().bfill())
    )

    # if units sold is NA then replace iwth 0
    df['units_sold'] = df['units_sold'].fillna(0)

    return df

# add a function to make sure our data types are correct
def correct_data_types(df):
    # DateTime conversion (handles NaT automatically)
    df['day'] = pd.to_datetime(df['day'])
    df['period_start_local'] = pd.to_datetime(df['period_start_local'])
    df['period_end_local'] = pd.to_datetime(df['period_end_local'])
    
    # Float conversions (handle NaN automatically)
    float_columns = ['regular_price', 'promo_price', 'avg_selling_price', 'unit_cost', 'avg_basket_size', 'total_basket_size', 'scraping_regular_price']
    for col in float_columns:
        df[col] = df[col].astype(float)
    
    # Integer conversions with null handling
    int_columns = ['units_sold', 'bb_id', 'period_id', 'merchant_id', 'store_id', 'item_id', 'num_baskets']
    for col in int_columns:
        # Only convert non-null values to int
        mask = df[col].notna()
        df.loc[mask, col] = df.loc[mask, col].astype(int)
        # Convert the entire column to nullable int
        df[col] = df[col].astype('Int64')
    
    return df

df_raw = df.copy()
time_start = time.time()
df_raw = generate_missing_rows(df_raw)
time_end = time.time()
print(f"Time taken for generate_missing_rows: {time_end - time_start} seconds")

time_start = time.time()
df_raw = fill_metadata_only(df_raw)
time_end = time.time()
print(f"Time taken for fill_metadata_only: {time_end - time_start} seconds")

time_start = time.time()
df_raw = correct_data_types(df_raw)
time_end = time.time()
print(f"Time taken for correct_data_types: {time_end - time_start} seconds")

time_start = time.time()
# insert this into table analysis_raw
write_to_table(df_raw, 'temporary', 'analysis_staging')
time_end = time.time()
print(f"Time taken for write_to_table: {time_end - time_start} seconds")
    

Time taken for generate_missing_rows: 10.217973947525024 seconds
Time taken for fill_metadata_only: 7.264764308929443 seconds
Time taken for correct_data_types: 2.011046886444092 seconds
Time taken for write_to_table: 35.9659698009491 seconds


## full cleaning for analysis_cleaned

In [30]:
# for situations of regular_price, promo_price, selling_price, unit_cost, if there is a null, then fill it with the last non-null value for taht item id / store_id combination 
import pandas as pd
import time
import numpy as np

def fill_null_prices_with_source(df):
    # 1) sort once
    df = df.sort_values(['item_id','store_id','day']).reset_index(drop=True)

    # 2) your lists
    fill_cols  = ['bb_id','timezone','period_id','merchant_id','name', 'period_start_local', 'period_end_local', 'on_promotion',
                  'regular_price','promo_price','avg_selling_price','unit_cost']
    track_cols = ['regular_price','promo_price','avg_selling_price','unit_cost']
    cols       = [c for c in fill_cols if c in df]

    # 3) copy originals for both tracking and thresholding
    orig = df[cols].copy()

    # 4a) do the “normal” fill for everything _except_ promo_price
    other_cols = [c for c in cols if c != 'promo_price']
    df[other_cols] = (
        df
        .groupby(['item_id','store_id'])[other_cols]
        .transform(lambda g: g.ffill().bfill())
    )

    # 4b) promo_price: only fill _within_ each (item,store,period) if ≥70% non‐null originally
    def _promo_fill(s):
        pct = s.notnull().sum() / len(s)
        if pct >= 0.7:
            return s.ffill().bfill()
        else:
            return s  # leave all as-is (no fill)
    df['promo_price'] = (
        df
        .groupby(['item_id','store_id','period_id'])['promo_price']
        .transform(_promo_fill)
    )
    # mark whether the fill or not happened as on_promotion flag 
    promo_eligible = (
        orig['promo_price']
        .notnull()
        .groupby([df['item_id'], df['store_id'], df['period_id']])
        .transform(lambda s: s.sum()/len(s) >= 0.7)
    )
    df['on_promotion'] = promo_eligible

    # 5) build mask of originally non-null (for all track_cols)
    mask = orig[track_cols].notnull().values  # shape (n,4)

    # 6) extract the “day” as numpy dates and broadcast
    days       = df['day'].values.astype('datetime64[D]')              # shape (n,)
    day_vals_2d = np.broadcast_to(days[:, None], mask.shape)           # shape (n,4)

    # 7) place src_vals where orig was non-null
    src_vals = np.full(mask.shape, np.datetime64('NaT'), dtype='datetime64[D]')
    src_vals[mask] = day_vals_2d[mask]

    # 8) make a DataFrame to run groupwise fills on those “source‐dates”
    src_df = (
        pd.DataFrame(src_vals, columns=track_cols)
          .assign(
             item_id   = df['item_id'],
             store_id  = df['store_id'],
             period_id = df['period_id']
          )
    )

    # 9a) for the non-promo columns, ffill→bfill _per item,store_ (as before)
    non_promo = [c for c in track_cols if c != 'promo_price']
    src_no_promo = (
        src_df
        .groupby(['item_id','store_id'])[non_promo]
        .transform(lambda g: g.ffill().bfill())
    )

    # 9b) for promo_price, ffill→bfill _per item,store,period_
    src_promo = (
        src_df
        .groupby(['item_id','store_id','period_id'])['promo_price']
        .transform(lambda g: g.ffill().bfill())
        .to_frame()
    )

    # 10) stitch back together
    src = pd.concat([
        src_no_promo,
        src_promo,
        src_df[['item_id','store_id']]
    ], axis=1)

    # 11) build your JSON snippets exactly as before
    snippets = []
    for col in track_cols:
        m = orig[col].isna() & src[col].notna()
        dates = src[col].dt.strftime('%Y-%m-%d').where(m, '')
        snippet = dates.str.replace(r'(.+)', f'"{col}":"\\1"', regex=True).where(m, '')
        snippets.append(snippet)

    snippet_df = pd.concat(snippets, axis=1)
    snippet_df.columns = track_cols

    # 12) join only the non-empty snippets into `{…}` or `{}` if none
    df['filled_from'] = snippet_df.apply(
        lambda row: '{' + ','.join(v for v in row if v) + '}',
        axis=1
    )

    # 13) your final Nan→0 fills
    if 'units_sold' in df:       df['units_sold']     = df['units_sold'].fillna(0)
    if 'avg_basket_size' in df:  df['avg_basket_size'] = df['avg_basket_size'].fillna(0)
    if 'total_basket_size' in df:df['total_basket_size']= df['total_basket_size'].fillna(0)
    if 'num_baskets' in df:      df['num_baskets']     = df['num_baskets'].fillna(0)

    return df

# add a function to make sure our data types are correct
def correct_data_types(df):
    # DateTime conversion (handles NaT automatically)
    df['day'] = pd.to_datetime(df['day'])
    df['period_start_local'] = pd.to_datetime(df['period_start_local'])
    df['period_end_local'] = pd.to_datetime(df['period_end_local'])

    # boolean conversion 
    df['on_promotion'] = df['on_promotion'].astype(bool)
    
    # Float conversions (handle NaN automatically)
    float_columns = ['regular_price', 'promo_price', 'avg_selling_price', 'unit_cost', 'avg_basket_size', 'total_basket_size', 'scraping_regular_price']
    for col in float_columns:
        df[col] = df[col].astype(float)
    
    # Integer conversions with null handling
    int_columns = ['units_sold', 'bb_id', 'period_id', 'merchant_id', 'store_id', 'item_id', 'num_baskets']
    for col in int_columns:
        # Only convert non-null values to int
        mask = df[col].notna()
        df.loc[mask, col] = df.loc[mask, col].astype(int)
        # Convert the entire column to nullable int
        df[col] = df[col].astype('Int64')
    
    return df

# for items that exist in a store_id on some dates but not all, we need to generate missing rows 
# for items in a store_id that exist after its first sold date populate all instances of units_sold = 0 for days that date doesnt exist in between start/now()
def generate_missing_rows(df):
    # 1) make sure 'day' is datetime (sanity check)
    df = df.copy()
    df['day'] = pd.to_datetime(df['day'])

    # 2) find the first date each item was store in a specific store_id
    sold_mask = df['units_sold'] > 0
    first_sold = (
        df[sold_mask]
        .groupby(['item_id', 'store_id'])['day']
        .min()
        .rename('first_sold')
    )

    # 3) get the overall last date
    last_date = df['day'].max()

    # 4) build the full calendar for each (item_id, store_id)
    records = []
    for (item_id, store_id), start_date in first_sold.items():
        # daily frequency from first sold to last_date
        full_days = pd.date_range(start=start_date, end=last_date, freq='D')
        records.extend(
            {'item_id': item_id, 'store_id': store_id, 'day': d}
            for d in full_days
        )
    full_calendar = pd.DataFrame(records)

    # 5) merge original data onto the full calendar
    df_complete = full_calendar.merge(
        df,
        on=['item_id', 'store_id', 'day'],
        how='left',
        suffixes=('', '_orig')
    )

    # backfilled information handelded in a different method

    return df_complete

def calculate_metrics(df):
    # active price is promo price if it on promotion, else regular price 
    df['active_price'] = np.where(df['on_promotion'],
                              df['promo_price'],
                              df['regular_price'])
    df['promo_spending'] = df['units_sold'] * df['regular_price'] / df['promo_price']
    df['revenue'] = df['units_sold'] * df['active_price']
    df['profit'] = df['units_sold'] * (df['active_price'] - df['unit_cost'])
    df['revenue_to_promo_spending_ratio'] = df['revenue'] / df['promo_spending']
    df['promo_spending (selling_price_based)'] = df['units_sold'] * df['regular_price'] / df['avg_selling_price']
    df['revenue (selling_price_based)'] = df['units_sold'] * df['avg_selling_price']
    df['profit (selling_price_based)'] = df['units_sold'] * (df['avg_selling_price'] - df['unit_cost'])
    df['revenue_to_promo_spending_ratio (selling_price_based)'] = df['revenue (selling_price_based)'] / df['promo_spending (selling_price_based)']
    
    return df

df_cleaning = df.copy()

time_start = time.time()
df_cleaning = generate_missing_rows(df_cleaning)
time_end = time.time()
print(f"Time taken to generate missing rows: {time_end - time_start} seconds")

time_start = time.time()
df_cleaned = fill_null_prices_with_source(df_cleaning)
time_end = time.time()
print(f"Time taken to fill null prices: {time_end - time_start} seconds")

time_start = time.time()
df_cleaned = correct_data_types(df_cleaned)
time_end = time.time()
print(f"Time taken to correct data types: {time_end - time_start} seconds")

time_start = time.time()
df_cleaned = calculate_metrics(df_cleaned)
time_end = time.time()
print(f"Time taken to calculate metrics: {time_end - time_start} seconds")

# order it a bit 
# reorder the columns 
df_cleaned = df_cleaned[['bb_id', 'period_id', 'period_start_local', 'period_end_local', 'on_promotion', 'merchant_id', 'store_id', 'item_id', 'name', 'day', 'timezone', 'active_price',
                         'regular_price', 'promo_price', 'avg_selling_price', 'unit_cost', 'units_sold', 
                         'total_basket_size', 'num_baskets', 'avg_basket_size', 
                         'promo_spending', 'revenue', 'profit', 'revenue_to_promo_spending_ratio', 
                         'promo_spending (selling_price_based)', 'revenue (selling_price_based)', 'profit (selling_price_based)', 'revenue_to_promo_spending_ratio (selling_price_based)', 
                         'filled_from']]
# Reset index if you want clean sequential indexing
df_cleaned = df_cleaned.reset_index(drop=True)

# Display the first 20 rows to verify
print(f"df num of rows: {df.shape[0]}, df_cleaned num of rows: {df_cleaned.shape[0]}, df_cleaned num of columns: {df_cleaned.shape[1]}")
df_cleaned.head(20) # almost 600 thousand rows 


Time taken to generate missing rows: 10.67543911933899 seconds
Time taken to fill null prices: 77.01239609718323 seconds
Time taken to correct data types: 2.417123794555664 seconds
Time taken to calculate metrics: 0.10974502563476562 seconds
df num of rows: 1996631, df_cleaned num of rows: 4447347, df_cleaned num of columns: 29


,bb_id,period_id,period_start_local,period_end_local,on_promotion,merchant_id,store_id,item_id,name,day,...,avg_basket_size,promo_spending,revenue,profit,revenue_to_promo_spending_ratio,promo_spending (selling_price_based),revenue (selling_price_based),profit (selling_price_based),revenue_to_promo_spending_ratio (selling_price_based),filled_from
0,8902,0,2024-06-21,2024-07-05,False,17,1173,8225800,Frozen Udon 2.75lb(1.25kg),2024-06-21,...,151.320,<NA>,5.99,4.0744,<NA>,1.111317,5.39,3.4744,4.8501,"{""unit_cost"":""2025-06-02""}"
1,8902,0,2024-06-21,2024-07-05,False,17,1173,8225800,Frozen Udon 2.75lb(1.25kg),2024-06-22,...,151.900,<NA>,5.99,4.0744,<NA>,1.111317,5.39,3.4744,4.8501,"{""unit_cost"":""2025-06-02""}"
2,8902,0,2024-06-21,2024-07-05,False,17,1173,8225800,Frozen Udon 2.75lb(1.25kg),2024-06-23,...,0.000,<NA>,0.0,0.0,<NA>,0.0,0.0,0.0,NaN,"{""regular_price"":""2024-06-22"",""avg_selling_pri..."
3,8902,0,2024-06-21,2024-07-05,False,17,1173,8225800,Frozen Udon 2.75lb(1.25kg),2024-06-24,...,0.000,<NA>,0.0,0.0,<NA>,0.0,0.0,0.0,NaN,"{""regular_price"":""2024-06-22"",""avg_selling_pri..."
4,8902,0,2024-06-21,2024-07-05,False,17,1173,8225800,Frozen Udon 2.75lb(1.25kg),2024-06-25,...,0.000,<NA>,0.0,0.0,<NA>,0.0,0.0,0.0,NaN,"{""regular_price"":""2024-06-22"",""avg_selling_pri..."
5,8902,0,2024-06-21,2024-07-05,False,17,1173,8225800,Frozen Udon 2.75lb(1.25kg),2024-06-26,...,0.000,<NA>,0.0,0.0,<NA>,0.0,0.0,0.0,NaN,"{""regular_price"":""2024-06-22"",""avg_selling_pri..."
6,8902,0,2024-06-21,2024-07-05,False,17,1173,8225800,Frozen Udon 2.75lb(1.25kg),2024-06-27,...,0.000,<NA>,0.0,0.0,<NA>,0.0,0.0,0.0,NaN,"{""regular_price"":""2024-06-22"",""avg_selling_pri..."
7,8902,0,2024-06-21,2024-07-05,False,17,1173,8225800,Frozen Udon 2.75lb(1.25kg),2024-06-28,...,0.000,<NA>,0.0,0.0,<NA>,0.0,0.0,0.0,NaN,"{""regular_price"":""2024-06-22"",""avg_selling_pri..."
8,8902,0,2024-06-21,2024-07-05,False,17,1173,8225800,Frozen Udon 2.75lb(1.25kg),2024-06-29,...,0.000,<NA>,0.0,0.0,<NA>,0.0,0.0,0.0,NaN,"{""regular_price"":""2024-06-22"",""avg_selling_pri..."
9,8902,0,2024-06-21,2024-07-05,False,17,1173,8225800,Frozen Udon 2.75lb(1.25kg),2024-06-30,...,0.000,<NA>,0.0,0.0,<NA>,0.0,0.0,0.0,NaN,"{""regular_price"":""2024-06-22"",""avg_selling_pri..."


In [31]:
# write into table 
time_start = time.time()
write_to_table(df_cleaned, 'temporary', 'analysis_cleaned')
time_end = time.time()
print(f"Time taken for write_to_table: {time_end - time_start} seconds")


Time taken for write_to_table: 65.10539412498474 seconds


In [20]:
# tion to calculate revenue per item per period, profit per item per period, promo spending per item per period based on these metrics 

def aggregate_data_by_period(df):
    df = (
        df
        .groupby(
            ['period_id', 'store_id', 'item_id'],
            as_index=False
        )
        .agg(
            avg_active_price    = ('active_price', 'mean'),
            avg_regular_price   = ('regular_price', 'mean'), 
            avg_promo_price      = ('promo_price',    'mean'),
            avg_selling_price   = ('avg_selling_price', 'mean'),
            avg_unit_cost       = ('unit_cost',     'mean'),
            total_units_sold    = ('units_sold',    'sum'),
            avg_basket_size     = ('avg_basket_size','sum'),
            total_num_baskets   = ('num_baskets', 'sum'),
            total_promo_spending = ('promo_spending','sum'),
            total_revenue       = ('revenue', 'sum'),
            total_profit        = ('profit', 'sum'),
            total_promo_spending_selling_price_based = ('promo_spending (selling_price_based)', 'sum'),
            total_revenue_selling_price_based = ('revenue (selling_price_based)', 'sum'),
            total_profit_selling_price_based = ('profit (selling_price_based)', 'sum'),
            # we are not going to aggregate filled from data here because that was row level 
            # non aggregations here
            name = ('name', 'first'),
            bb_id = ('bb_id', 'first'),
            merchant_id = ('merchant_id', 'first'),
            timezone = ('timezone', 'first'),
            period_start_local = ('period_start_local', 'first'),
            period_end_local = ('period_end_local', 'first')

            
        )
    )
    df['weighted_average_promo_spending_ratio'] = df['total_promo_spending'] / df['total_revenue']
    df['weighted_average_promo_spending_ratio_selling_price_based'] = df['total_promo_spending_selling_price_based'] / df['total_revenue_selling_price_based']
    return df

df_analyzed = aggregate_data_by_period(df_cleaned)

# # order by total_profit descending
df_analyzed = df_analyzed.sort_values('total_profit', ascending=False)
df_analyzed.head(20)


# df_test = df.copy()
# df_test = df_test[(df_test['item_id'] == 8229321)]
# df_test = generate_missing_rows(df_test)
# df_test = fill_null_prices_with_source(df_test)
# df_test = correct_data_types(df_test)
# df_test = calculate_metrics(df_test)
# # only keep 
# # filter out where promo_price is not nan

# df_test.head(20)

# filter for 8226315 item_id and periods 22 and 23

    

,period_id,store_id,item_id,avg_active_price,avg_regular_price,avg_promo_price,avg_selling_price,avg_unit_cost,total_units_sold,avg_basket_size,...,total_revenue_selling_price_based,total_profit_selling_price_based,name,bb_id,merchant_id,timezone,period_start_local,period_end_local,weighted_average_promo_spending_ratio,weighted_average_promo_spending_ratio_selling_price_based
5225,1,1173,8226315,14.990000,14.990000,NaN,9.309523,6.0847,420,1438.082593,...,2647.5,91.926,Shin Ramen 4.23oz(120g) 10 Packs,1764,17,America/New_York,2024-07-05 00:00:00,2024-07-19 00:00:00,0.0,0.382207
20322,4,1173,8225846,15.990000,15.990000,12.99,13.750723,11.0000,570,2211.679041,...,7452.4,1182.4,Cooked White Rice Box 7.4oz(210g) 12 Ea,3437,17,America/New_York,2024-08-16 00:00:00,2024-08-30 00:00:00,0.049971,0.093789
72158,13,1173,8226147,45.990000,45.990000,29.99,32.575281,24.9974,128,890.306000,...,3623.12,423.4528,Korean Rice Suhyang Rice 22lb(10kg),7689,17,America/New_York,2024-12-19 23:00:00,2025-01-02 23:00:00,0.024487,0.057602
92121,16,1173,8228198,283.930000,283.930000,39.99,38.104286,17.6410,10,658.147500,...,363.9,187.49,Coarse Red Pepper Powder 2.2lb(1kg),7698,17,America/New_York,2025-01-30 23:00:00,2025-02-13 23:00:00,0.010003,0.215304
20311,4,1173,8225835,18.990000,18.990000,13.99,13.457053,9.6864,285,1759.167015,...,3710.05,949.426,Shin Ramen 16Pk Box 4.23OZ(120G)*16,1801,17,America/New_York,2024-08-16 00:00:00,2024-08-30 00:00:00,0.022071,0.112443
20312,4,1173,8225836,6.790000,6.790000,5.22,4.464164,3.5750,820,2103.843768,...,3164.95,233.45,Bibim Men Spicy Cold Noodles 4.58oz(130g) 5 Pack,7077,17,America/New_York,2024-08-16 00:00:00,2024-08-30 00:00:00,0.115942,0.461943
60430,11,1173,8225824,14.990000,14.990000,10.99,11.109415,7.6900,354,2259.164543,...,3670.06,947.8,Beef Bone Soup 16.9 fl.oz(500ml) 6 Packs,4070,17,America/New_York,2024-11-21 23:00:00,2024-12-05 23:00:00,0.051922,0.141443
60745,11,1173,8226238,14.990000,14.990000,10.99,10.867881,6.6094,290,1815.188512,...,2991.5,1074.774,Cooked White Rice 7.4oz(210g) 12 Packs,3996,17,America/New_York,2024-11-21 23:00:00,2024-12-05 23:00:00,0.046123,0.141972
68990,12,1173,8232131,12.990000,22.990000,12.99,11.972066,8.7500,749,1765.486021,...,8695.91,2142.16,Tokyo Banana 1.06LB (480G),2393886,17,America/New_York,2024-12-05 23:00:00,2024-12-19 23:00:00,0.164354,0.172554
26390,5,1173,8225846,15.990000,15.990000,12.99,12.259917,11.0000,474,2028.412436,...,5771.2,557.2,Cooked White Rice Box 7.4oz(210g) 12 Ea,3437,17,America/New_York,2024-08-30 00:00:00,2024-09-13 00:00:00,0.009907,0.108278


In [96]:
# quick analysis of hmart file
file_path = "/Users/kappavi/Documents/Avi's Documents /Work/data/hmart/new_daily list.csv"
# file_path = "/Users/kappavi/Documents/Avi's Documents /Work/data/hmart/Daily sales list copy.csv"
# file_path = "/Users/kappavi/Documents/Avi's Documents /Work/data/hmart/Report_transformed.csv"
# load csv into df
df_hmart_debug = pd.read_csv(file_path)
df_hmart_debug.head()

print(df_hmart_debug.dtypes)

# filter based on last change date of 2025-02-19 and ID SKU 111
# df_hmart_debug = df_hmart_debug[df_hmart_debug['UPC'] == 3114602398]
# filter based on name containing "Shin Ramen"
df_hmart_debug = df_hmart_debug[df_hmart_debug['SKU Name'].str.contains('Shin Ramen 4.23oz')]
# date should be between 2/17 and 2/20
df_hmart_debug = df_hmart_debug[(df_hmart_debug['Creation Date'] >= '2025-02-17') & (df_hmart_debug['Creation Date'] <= '2025-02-21')]
df_hmart_debug = df_hmart_debug[['SKU Name', 'Creation Date', 'Last Change Date', 'SKU Value']]
# order by last change date
df_hmart_debug = df_hmart_debug.sort_values('Creation Date')

df_hmart_debug.head(20)

df_hmart_debug_2 = pd.read_csv(file_path)
# want to group by SKU Name, and then see the creation date and last change date 
# order by creation date
df_hmart_debug_2 = df_hmart_debug_2.sort_values('Creation Date')
# show sku name, sku value, creation date
df_hmart_debug_2 = df_hmart_debug_2[['SKU Name', 'SKU Value', 'Creation Date']]
# find SKU value of 14.13
df_hmart_debug_2 = df_hmart_debug_2[df_hmart_debug_2['SKU Value'] == 14.13]

df_hmart_debug_2.head(20)








/var/folders/hx/4z3d1xsd4kn2grfhp_cvzcm80000gn/T/ipykernel_58731/4210629795.py:6: DtypeWarning: Columns (14,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df_hmart_debug = pd.read_csv(file_path)


Origin                             object
Order                              object
Sequence                            int64
Creation Date                      object
Courrier                           object
Estimate Delivery Date             object
Delivery Deadline                  object
Status                             object
Last Change Date                   object
UtmMedium                          object
Quantity_SKU                        int64
ID_SKU                              int64
Category Ids Sku                   object
Reference Code                      int64
UPC                                object
SKU Name                           object
SKU Value                         float64
SKU Selling Price                 float64
SKU Total Price                   float64
SKU Path                           object
Item Attachments                  float64
List Id                           float64
List Type Name                    float64
Service (Price/ Selling Price)    

/var/folders/hx/4z3d1xsd4kn2grfhp_cvzcm80000gn/T/ipykernel_58731/4210629795.py:23: DtypeWarning: Columns (14,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df_hmart_debug_2 = pd.read_csv(file_path)


,SKU Name,SKU Value,Creation Date


In [27]:
# lets do some manual analysis here

# lets compare for example period 25 and 24 
# lets find top 10 revenue grossing items for period 25 and 24 
df_24 = df_analyzed[df_analyzed['period_id'] == 24]
df_25 = df_analyzed[df_analyzed['period_id'] == 25]

df_24.head(20)
df_25.head(20)

# lets find the top 10 revenue grossing items for period 25 and 24 
df_24_top_10 = df_24.sort_values('total_revenue', ascending=False).head(10)
df_25_top_10 = df_25.sort_values('total_revenue', ascending=False).head(10)

df_24_top_10.head(20)
df_25_top_10.head(20)

# lets find top 10 items based on revenue_to_promo_spending_ratio for period 25 and 24 
df_24_top_10_revenue_to_promo_spending_ratio = df_24.sort_values('weighted_average_promo_spending_ratio', ascending=False).head(10)
df_25_top_10_revenue_to_promo_spending_ratio = df_25.sort_values('weighted_average_promo_spending_ratio', ascending=False).head(10)

df_24_top_10_revenue_to_promo_spending_ratio.head(20)
df_25_top_10_revenue_to_promo_spending_ratio.head(20)

# lets find top 10 items that have greatest units sold 
df_24_top_10_units_sold = df_24.sort_values('total_units_sold', ascending=False).head(10)
df_25_top_10_units_sold = df_25.sort_values('total_units_sold', ascending=False).head(10)

df_24_top_10_units_sold.head(20)
df_25_top_10_units_sold.head(20)

# create a df that melts this based on period
# common: bb_id, period_id, merchant_id, store_id, item_id, name
# 24: revenue, profit, promo_spending, total_units_sold, revenue_to_promo_spending_ratio
# 25: revenue, profit, promo_spending, total_units_sold, revenue_to_promo_spending_ratio
keys    = ['bb_id','merchant_id','store_id','item_id','name']
metrics = [
    'total_revenue',
    'total_profit',
    'total_promo_spending',
    'total_units_sold',
    'weighted_average_promo_spending_ratio'
]

# slice each period down to keys + metrics
df24 = df_24[keys + metrics]
df25 = df_25[keys + metrics]

# now merge them side-by-side, adding suffixes for period 24 vs 25
df_compare = pd.merge(
    df24,
    df25,
    on=keys,
    how='inner',
    suffixes=('_24','_25')
)
# add a metric for lift from 24 to 25 
df_compare['lift'] = df_compare['total_revenue_25'] / df_compare['total_revenue_24']
# order by lift
df_compare = df_compare.sort_values('lift', ascending=False)
df_compare.head(20)



,bb_id,merchant_id,store_id,item_id,name,total_revenue_24,total_profit_24,total_promo_spending_24,total_units_sold_24,weighted_average_promo_spending_ratio_24,total_revenue_25,total_profit_25,total_promo_spending_25,total_units_sold_25,weighted_average_promo_spending_ratio_25,lift
3315,7625,17,1235,8229779,Coarse Red Pepper Powder 3lb(1.36kg) for Kimchi,0.0,0.0,0.0,0,NaN,0.0,0.0,0.0,0,NaN,NaN
2961,2356,17,1235,8228777,Dark Chocolate 8.82oz(250g),0.0,0.0,0.0,1,NaN,0.0,0.0,0.0,0,NaN,NaN
2947,800180,17,1235,8228744,Matcha Green Tea Traditional Tea Bags 0.05oz(1...,0.0,0.0,0.0,0,NaN,0.0,0.0,0.0,0,NaN,NaN
2948,2428324,17,1235,8228747,Hong Kong Style Milk Tea 8.45 fl.oz(250ml) X 6...,0.0,0.0,0.0,1,NaN,0.0,0.0,0.0,1,NaN,NaN
2950,7134,17,1235,8228783,European Premium Roll Type Cookies 3.7oz(105g),0.0,0.0,0.0,0,NaN,0.0,0.0,0.0,0,NaN,NaN
2952,3673,17,1235,8228787,Black Milk Tea 11.8 fl.oz(350ml),0.0,0.0,0.0,0,NaN,0.0,0.0,0.0,0,NaN,NaN
2953,8024,17,1235,8228750,Konjac Jelly Collagen Watermelon10Pk 50.70fl(1...,0.0,0.0,0.0,0,NaN,0.0,0.0,0.0,0,NaN,NaN
2954,3974,17,1235,8228794,Nature Black Rice 2lb (0.9kg),0.0,0.0,0.0,0,NaN,0.0,0.0,0.0,0,NaN,NaN
2955,7791,17,1235,8228797,Hallabong Tea 2.09lb(950g),0.0,0.0,0.0,0,NaN,0.0,0.0,0.0,0,NaN,NaN
2956,7030,17,1235,8228798,Choco Boy Big Size 5.07oz(144g),0.0,0.0,0.0,0,NaN,0.0,0.0,0.0,0,NaN,NaN


In [167]:
import pandas as pd
import plotly.express as px
from IPython.display import display
import ipywidgets as widgets
import nbformat
print(nbformat.__version__)  # should be ≥ 4.2.0

# Ensure `df_compare` is defined in your notebook's environment
# df_compare = pd.merge(...)

# Define which metrics to explore
metrics = [
    'revenue_24', 'revenue_25',
    'profit_24', 'profit_25',
    'total_promo_spending_24', 'total_promo_spending_25',
    'total_units_sold_24', 'total_units_sold_25',
    'revenue_to_promo_spending_ratio_24', 'revenue_to_promo_spending_ratio_25',
    'lift'
]

# Widgets for selecting metric and number of top items
metric_dropdown = widgets.Dropdown(options=metrics, description='Metric:')
top_n_slider = widgets.IntSlider(value=10, min=1, max=50, step=1, description='Top N:')

# Update function to redraw chart based on selections
def update_plot(metric, top_n):
    # Select top N rows by chosen metric
    df_plot = df_compare.nlargest(top_n, metric)
    # Create bar chart
    fig = px.bar(
        df_plot,
        x='name',
        y=metric,
        title=f'Top {top_n} Items by {metric.replace("_", " ")}',
        labels={metric: metric}
    )
    fig.update_layout(
        xaxis_tickangle=-45,
        yaxis_title=metric
    )
    fig.show()

# Assemble interactive dashboard
dashboard = widgets.interactive(update_plot, metric=metric_dropdown, top_n=top_n_slider)

display(dashboard)


5.10.4


interactive(children=(Dropdown(description='Metric:', options=('revenue_24', 'revenue_25', 'profit_24', 'profi…

# trying some even greater analysis

In [127]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime

# Set matplotlib style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

class PriceTrackingDashboard:
    def __init__(self, df_analyzed):
        self.df_analyzed = df_analyzed
        self.filtered_data = None
        self.analysis_data = None
        
        # Get unique values for filters
        self.periods = sorted(df_analyzed['period_id'].unique())
        self.items = sorted(df_analyzed['name'].unique())
        self.stores = sorted(df_analyzed['store_id'].unique())
        self.merchants = sorted(df_analyzed['merchant_id'].unique())
        
        # Create widgets
        self.create_widgets()
        
        # Create output widget for plots
        self.output = widgets.Output()
        
        # Set up interactions
        self.setup_interactions()
        
        # Initial update
        self.update_dashboard()
    
    def create_widgets(self):
        """Create all interactive widgets"""
        
        # Period selection
        self.current_period = widgets.Dropdown(
            options=[(f'Period {p}', p) for p in self.periods],
            value=self.periods[-1] if self.periods else None,
            description='Current Period:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='250px')
        )
        
        self.comparison_period = widgets.Dropdown(
            options=[(f'Period {p}', p) for p in self.periods],
            value=self.periods[-2] if len(self.periods) > 1 else (self.periods[0] if self.periods else None),
            description='Comparison Period:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='250px')
        )
        
        # Filters
        self.item_filter = widgets.SelectMultiple(
            options=self.items,
            value=self.items[:10] if len(self.items) > 10 else self.items,
            description='Items:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px', height='120px')
        )
        
        self.store_filter = widgets.SelectMultiple(
            options=[(f'Store {s}', s) for s in self.stores],
            value=self.stores,
            description='Stores:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='250px', height='120px')
        )
        
        self.merchant_filter = widgets.SelectMultiple(
            options=[(f'Merchant {m}', m) for m in self.merchants],
            value=self.merchants,
            description='Merchants:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='250px', height='120px')
        )
        
        # Sort and display options
        self.sort_by = widgets.Dropdown(
            options=[
                ('Revenue (Highest)', 'revenue_desc'),
                ('Revenue (Lowest)', 'revenue_asc'),
                ('Profit (Highest)', 'profit_desc'),
                ('Profit (Lowest)', 'profit_asc'),
                ('Revenue Variance', 'revenue_variance_desc'),
                ('Profit Variance', 'profit_variance_desc'),
                ('Sales Lift', 'sales_lift_desc')
            ],
            value='revenue_desc',
            description='Sort By:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='250px')
        )
        
        self.top_n = widgets.IntSlider(
            value=20,
            min=5,
            max=50,
            step=5,
            description='Top N Items:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        # Chart selection
        self.chart_type = widgets.Dropdown(
            options=[
                ('Revenue vs Profit', 'revenue_profit'),
                ('Variance Analysis', 'variance'),
                ('Sales Lift', 'lift'),
                ('Performance Matrix', 'matrix'),
                ('Summary Table', 'table')
            ],
            value='revenue_profit',
            description='Chart Type:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='250px')
        )
        
        # Update button
        self.update_button = widgets.Button(
            description='Update Dashboard',
            button_style='primary',
            layout=widgets.Layout(width='200px')
        )
    
    def setup_interactions(self):
        """Set up widget interactions"""
        self.update_button.on_click(lambda b: self.update_dashboard())
        
        # Auto-update on certain widget changes
        for widget in [self.current_period, self.comparison_period, self.chart_type]:
            widget.observe(lambda change: self.update_dashboard(), names='value')
    
    def calculate_variance_and_lift(self, df, current_period, comparison_period):
        """Calculate variance and lift between periods"""
        current_data = df[df['period_id'] == current_period].copy()
        comparison_data = df[df['period_id'] == comparison_period].copy()
        
        # Merge on item/store level
        merged = current_data.merge(
            comparison_data[['bb_id', 'store_id', 'item_id', 'revenue', 'profit', 'total_units_sold']], 
            on=['bb_id', 'store_id', 'item_id'], 
            how='left', 
            suffixes=('_current', '_comparison')
        )
        
        # Fill NaN values
        merged = merged.fillna(0)
        
        # Calculate variance and lift
        merged['revenue_variance'] = merged['revenue_current'] - merged['revenue_comparison']
        merged['profit_variance'] = merged['profit_current'] - merged['profit_comparison']
        merged['revenue_variance_pct'] = np.where(merged['revenue_comparison'] > 0, 
                                                (merged['revenue_variance'] / merged['revenue_comparison']) * 100, 0)
        merged['profit_variance_pct'] = np.where(merged['profit_comparison'] > 0, 
                                               (merged['profit_variance'] / merged['profit_comparison']) * 100, 0)
        
        # Calculate lift (sales ratio)
        merged['sales_lift'] = np.where(merged['total_units_sold_comparison'] > 0, 
                                       merged['total_units_sold_current'] / merged['total_units_sold_comparison'], 0)
        
        return merged
    
    def filter_and_sort_data(self):
        """Filter and sort data based on widget values"""
        # Filter data
        filtered_df = self.df_analyzed.copy()
        
        if self.item_filter.value:
            filtered_df = filtered_df[filtered_df['name'].isin(self.item_filter.value)]
        if self.store_filter.value:
            filtered_df = filtered_df[filtered_df['store_id'].isin(self.store_filter.value)]
        if self.merchant_filter.value:
            filtered_df = filtered_df[filtered_df['merchant_id'].isin(self.merchant_filter.value)]
        
        # Calculate variance and lift
        if self.current_period.value and self.comparison_period.value:
            analysis_df = self.calculate_variance_and_lift(filtered_df, self.current_period.value, self.comparison_period.value)
        else:
            analysis_df = filtered_df[filtered_df['period_id'] == self.current_period.value].copy()
            analysis_df['revenue_variance'] = 0
            analysis_df['profit_variance'] = 0
            analysis_df['sales_lift'] = 1
        
        # Sort data
        sort_mapping = {
            'revenue_desc': ('revenue_current', False),
            'revenue_asc': ('revenue_current', True),
            'profit_desc': ('profit_current', False),
            'profit_asc': ('profit_current', True),
            'revenue_variance_desc': ('revenue_variance', False),
            'profit_variance_desc': ('profit_variance', False),
            'sales_lift_desc': ('sales_lift', False)
        }
        
        if self.sort_by.value in sort_mapping:
            sort_col, ascending = sort_mapping[self.sort_by.value]
            analysis_df = analysis_df.sort_values(sort_col, ascending=ascending)
        
        # Limit to top N
        analysis_df = analysis_df.head(self.top_n.value)
        
        self.filtered_data = filtered_df
        self.analysis_data = analysis_df
        
        return analysis_df
    
    def create_summary_stats(self, analysis_df):
        """Create summary statistics"""
        total_revenue_current = analysis_df['revenue_current'].sum()
        total_profit_current = analysis_df['profit_current'].sum()
        total_revenue_variance = analysis_df['revenue_variance'].sum()
        total_profit_variance = analysis_df['profit_variance'].sum()
        avg_lift = analysis_df['sales_lift'].mean()
        
        print("="*80)
        print("DASHBOARD SUMMARY")
        print("="*80)
        print(f"📊 Total Revenue (Current):     ${total_revenue_current:,.0f}")
        print(f"💰 Total Profit (Current):      ${total_profit_current:,.0f}")
        print(f"📈 Revenue Variance:            ${total_revenue_variance:,.0f}")
        print(f"📈 Profit Variance:             ${total_profit_variance:,.0f}")
        print(f"🚀 Average Sales Lift:          {avg_lift:.2f}x")
        print(f"📦 Number of Items Analyzed:    {len(analysis_df)}")
        print("="*80)
    
    def plot_revenue_profit(self, analysis_df):
        """Create revenue vs profit chart"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # Revenue and Profit Bar Chart
        items = analysis_df['name'].head(10)
        x_pos = np.arange(len(items))
        
        ax1.bar(x_pos - 0.2, analysis_df['revenue_current'].head(10), 0.4, 
                label='Revenue', alpha=0.8, color='steelblue')
        ax1.bar(x_pos + 0.2, analysis_df['profit_current'].head(10), 0.4, 
                label='Profit', alpha=0.8, color='orange')
        
        ax1.set_xlabel('Items')
        ax1.set_ylabel('Amount ($)')
        ax1.set_title('Revenue vs Profit (Top 10 Items)')
        ax1.set_xticks(x_pos)
        ax1.set_xticklabels([name[:20] + '...' if len(name) > 20 else name for name in items], 
                           rotation=45, ha='right')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Profit Margin Analysis
        analysis_df_copy = analysis_df.copy()
        analysis_df_copy['profit_margin'] = np.where(analysis_df_copy['revenue_current'] > 0,
                                                   analysis_df_copy['profit_current'] / analysis_df_copy['revenue_current'],
                                                   0)
        
        scatter = ax2.scatter(analysis_df_copy['revenue_current'], analysis_df_copy['profit_current'],
                             s=analysis_df_copy['sales_lift'] * 50, alpha=0.6, c=analysis_df_copy['profit_margin'],
                             cmap='RdYlGn')
        
        ax2.set_xlabel('Revenue ($)')
        ax2.set_ylabel('Profit ($)')
        ax2.set_title('Revenue vs Profit (Bubble size = Sales Lift)')
        ax2.grid(True, alpha=0.3)
        
        # Add colorbar
        cbar = plt.colorbar(scatter, ax=ax2)
        cbar.set_label('Profit Margin')
        
        plt.tight_layout()
        plt.show()
    
    def plot_variance_analysis(self, analysis_df):
        """Create variance analysis charts"""
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10))
        
        # Revenue and Profit Variance
        items = analysis_df['name'].head(15)
        x_pos = np.arange(len(items))
        
        colors_rev = ['green' if x >= 0 else 'red' for x in analysis_df['revenue_variance'].head(15)]
        colors_prof = ['green' if x >= 0 else 'red' for x in analysis_df['profit_variance'].head(15)]
        
        ax1.bar(x_pos, analysis_df['revenue_variance'].head(15), color=colors_rev, alpha=0.7)
        ax1.set_xlabel('Items')
        ax1.set_ylabel('Revenue Variance ($)')
        ax1.set_title('Revenue Variance by Item (Top 15)')
        ax1.set_xticks(x_pos)
        ax1.set_xticklabels([name[:15] + '...' if len(name) > 15 else name for name in items], 
                           rotation=45, ha='right')
        ax1.axhline(y=0, color='black', linestyle='-', alpha=0.5)
        ax1.grid(True, alpha=0.3)
        
        ax2.bar(x_pos, analysis_df['profit_variance'].head(15), color=colors_prof, alpha=0.7)
        ax2.set_xlabel('Items')
        ax2.set_ylabel('Profit Variance ($)')
        ax2.set_title('Profit Variance by Item (Top 15)')
        ax2.set_xticks(x_pos)
        ax2.set_xticklabels([name[:15] + '...' if len(name) > 15 else name for name in items], 
                           rotation=45, ha='right')
        ax2.axhline(y=0, color='black', linestyle='-', alpha=0.5)
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def plot_sales_lift(self, analysis_df):
        """Create sales lift analysis"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # Sales Lift Bar Chart
        items = analysis_df['name'].head(15)
        lifts = analysis_df['sales_lift'].head(15)
        colors = ['green' if x >= 1 else 'red' for x in lifts]
        
        ax1.bar(range(len(items)), lifts, color=colors, alpha=0.7)
        ax1.set_xlabel('Items')
        ax1.set_ylabel('Sales Lift (Ratio)')
        ax1.set_title('Sales Lift by Item (Top 15)')
        ax1.set_xticks(range(len(items)))
        ax1.set_xticklabels([name[:15] + '...' if len(name) > 15 else name for name in items], 
                           rotation=45, ha='right')
        ax1.axhline(y=1, color='black', linestyle='--', alpha=0.7, label='Baseline (1.0x)')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Sales Lift Distribution
        ax2.hist(analysis_df['sales_lift'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
        ax2.axvline(x=1, color='red', linestyle='--', alpha=0.7, label='Baseline (1.0x)')
        ax2.axvline(x=analysis_df['sales_lift'].mean(), color='orange', linestyle='-', 
                   alpha=0.7, label=f'Mean ({analysis_df["sales_lift"].mean():.2f}x)')
        ax2.set_xlabel('Sales Lift')
        ax2.set_ylabel('Frequency')
        ax2.set_title('Sales Lift Distribution')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def plot_performance_matrix(self, analysis_df):
        """Create performance matrix"""
        fig, ax = plt.subplots(figsize=(12, 8))
        
        # Create quadrant analysis
        revenue_median = analysis_df['revenue_current'].median()
        profit_median = analysis_df['profit_current'].median()
        
        # Color code by quadrant
        colors = []
        for _, row in analysis_df.iterrows():
            if row['revenue_current'] >= revenue_median and row['profit_current'] >= profit_median:
                colors.append('green')  # High revenue, high profit
            elif row['revenue_current'] >= revenue_median and row['profit_current'] < profit_median:
                colors.append('orange')  # High revenue, low profit
            elif row['revenue_current'] < revenue_median and row['profit_current'] >= profit_median:
                colors.append('blue')  # Low revenue, high profit
            else:
                colors.append('red')  # Low revenue, low profit
        
        scatter = ax.scatter(analysis_df['revenue_current'], analysis_df['profit_current'],
                           s=analysis_df['sales_lift'] * 100, alpha=0.6, c=colors)
        
        # Add quadrant lines
        ax.axvline(x=revenue_median, color='black', linestyle='--', alpha=0.5)
        ax.axhline(y=profit_median, color='black', linestyle='--', alpha=0.5)
        
        # Add quadrant labels
        ax.text(analysis_df['revenue_current'].max() * 0.8, analysis_df['profit_current'].max() * 0.9,
                'High Rev\nHigh Profit', ha='center', va='center', fontsize=10, 
                bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen", alpha=0.7))
        
        ax.text(analysis_df['revenue_current'].max() * 0.8, analysis_df['profit_current'].max() * 0.1,
                'High Rev\nLow Profit', ha='center', va='center', fontsize=10,
                bbox=dict(boxstyle="round,pad=0.3", facecolor="orange", alpha=0.7))
        
        ax.text(analysis_df['revenue_current'].max() * 0.2, analysis_df['profit_current'].max() * 0.9,
                'Low Rev\nHigh Profit', ha='center', va='center', fontsize=10,
                bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.7))
        
        ax.text(analysis_df['revenue_current'].max() * 0.2, analysis_df['profit_current'].max() * 0.1,
                'Low Rev\nLow Profit', ha='center', va='center', fontsize=10,
                bbox=dict(boxstyle="round,pad=0.3", facecolor="lightcoral", alpha=0.7))
        
        ax.set_xlabel('Revenue ($)')
        ax.set_ylabel('Profit ($)')
        ax.set_title('Performance Matrix (Bubble size = Sales Lift)')
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def show_summary_table(self, analysis_df):
        """Display summary table"""
        # Select key columns for display
        display_cols = ['name', 'store_id', 'revenue_current', 'profit_current', 
                       'revenue_variance', 'profit_variance', 'sales_lift']
        
        table_df = analysis_df[display_cols].copy()
        table_df.columns = ['Item Name', 'Store ID', 'Current Revenue', 'Current Profit',
                           'Revenue Variance', 'Profit Variance', 'Sales Lift']
        
        # Format currency columns
        for col in ['Current Revenue', 'Current Profit', 'Revenue Variance', 'Profit Variance']:
            table_df[col] = table_df[col].apply(lambda x: f'${x:,.0f}')
        
        table_df['Sales Lift'] = table_df['Sales Lift'].apply(lambda x: f'{x:.2f}x')
        
        print("\nDETAILED ANALYSIS TABLE")
        print("="*120)
        print(table_df.to_string(index=False, max_colwidth=30))
    
    def update_dashboard(self):
        """Update the entire dashboard"""
        with self.output:
            clear_output(wait=True)
            
            # Filter and sort data
            analysis_df = self.filter_and_sort_data()
            
            if analysis_df.empty:
                print("No data available with current filters.")
                return
            
            # Show summary statistics
            self.create_summary_stats(analysis_df)
            
            # Create visualizations based on selected chart type
            if self.chart_type.value == 'revenue_profit':
                self.plot_revenue_profit(analysis_df)
            elif self.chart_type.value == 'variance':
                self.plot_variance_analysis(analysis_df)
            elif self.chart_type.value == 'lift':
                self.plot_sales_lift(analysis_df)
            elif self.chart_type.value == 'matrix':
                self.plot_performance_matrix(analysis_df)
            elif self.chart_type.value == 'table':
                self.show_summary_table(analysis_df)
    
    def display(self):
        """Display the complete dashboard"""
        # Create layout
        period_box = widgets.HBox([self.current_period, self.comparison_period])
        filter_box = widgets.HBox([self.item_filter, self.store_filter, self.merchant_filter])
        control_box = widgets.HBox([self.sort_by, self.top_n, self.chart_type])
        
        dashboard_layout = widgets.VBox([
            widgets.HTML("<h2>📊 Price Tracking Performance Dashboard</h2>"),
            widgets.HTML("<h3>Period Selection</h3>"),
            period_box,
            widgets.HTML("<h3>Filters</h3>"),
            filter_box,
            widgets.HTML("<h3>Display Options</h3>"),
            control_box,
            self.update_button,
            self.output
        ])
        
        display(dashboard_layout)

# Usage example:
dashboard = PriceTrackingDashboard(df_analyzed)
dashboard.display()

In [100]:
import pandas as pd
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Replace this with your actual aggregated DataFrame
df = df_analyzed.copy()

# Unique values for filters
periods   = sorted(df['period_id'].unique())
items     = sorted(df['item_id'].unique())
stores    = sorted(df['store_id'].unique())
merchants = sorted(df['merchant_id'].unique())

app = Dash(__name__)

app.layout = html.Div([
    html.H1("Interactive Promotions Dashboard"),
    html.Div([
        html.Div([
            html.Label("Current Period"),
            dcc.Dropdown(
                id='current-period',
                options=[{'label': p, 'value': p} for p in periods],
                value=periods[-1]
            ),
        ], style={'width': '30%', 'display': 'inline-block'}),
        html.Div([
            html.Label("Promo Period"),
            dcc.Dropdown(
                id='promo-period',
                options=[{'label': p, 'value': p} for p in periods],
                value=periods[0]
            ),
        ], style={'width': '30%', 'display': 'inline-block', 'marginLeft': '2%'}),
    ]),
    html.Div([
        html.Div([
            html.Label("Item"),
            dcc.Dropdown(
                id='item-filter',
                options=[{'label': i, 'value': i} for i in items],
                placeholder="All Items"
            ),
        ], style={'width': '30%', 'display': 'inline-block'}),
        html.Div([
            html.Label("Store"),
            dcc.Dropdown(
                id='store-filter',
                options=[{'label': s, 'value': s} for s in stores],
                placeholder="All Stores"
            ),
        ], style={'width': '30%', 'display': 'inline-block', 'marginLeft': '2%'}),
        html.Div([
            html.Label("Merchant"),
            dcc.Dropdown(
                id='merchant-filter',
                options=[{'label': m, 'value': m} for m in merchants],
                placeholder="All Merchants"
            ),
        ], style={'width': '30%', 'display': 'inline-block', 'marginLeft': '2%'}),
    ], style={'marginTop': '20px'}),
    dcc.Graph(id='revenue-diff'),
    dcc.Graph(id='profit-diff'),
    dcc.Graph(id='lift-chart'),
    dcc.Graph(id='top-revenue-items'),
    dcc.Graph(id='top-profit-items')
])

@app.callback(
    [
        Output('revenue-diff', 'figure'),
        Output('profit-diff', 'figure'),
        Output('lift-chart', 'figure'),
        Output('top-revenue-items', 'figure'),
        Output('top-profit-items', 'figure')
    ],
    [
        Input('current-period', 'value'),
        Input('promo-period', 'value'),
        Input('item-filter', 'value'),
        Input('store-filter', 'value'),
        Input('merchant-filter', 'value')
    ]
)
def update_charts(current_period, promo_period, item_id, store_id, merchant_id):
    d = df.copy()
    # Apply filters
    if item_id:
        d = d[d['item_id'] == item_id]
    if store_id:
        d = d[d['store_id'] == store_id]
    if merchant_id:
        d = d[d['merchant_id'] == merchant_id]

    # Split periods
    df_curr  = d[d['period_id'] == current_period]
    df_promo = d[d['period_id'] == promo_period]

    # Merge on item/store/merchant
    merged = pd.merge(
        df_curr, df_promo,
        on=['item_id','store_id','merchant_id'],
        suffixes=('_curr', '_promo')
    )

    # Calculate diffs & lift
    merged['rev_diff']    = merged['revenue_curr'] - merged['revenue_promo']
    merged['profit_diff'] = merged['profit_curr']  - merged['profit_promo']
    merged['lift']        = merged['total_units_sold_curr'] / merged['total_units_sold_promo']

    # Build figures
    fig_rev_diff = px.bar(
        merged, x='item_id', y='rev_diff',
        labels={'rev_diff':'Revenue Δ'},
        title="Revenue Change (Current vs Promo)"
    )
    fig_profit_diff = px.bar(
        merged, x='item_id', y='profit_diff',
        labels={'profit_diff':'Profit Δ'},
        title="Profit Change (Current vs Promo)"
    )
    fig_lift = px.bar(
        merged, x='item_id', y='lift',
        labels={'lift':'Sales Lift'},
        title="Sales Lift Ratio"
    )

    top_rev = df_curr.groupby('item_id', as_index=False)['revenue'].sum().nlargest(10,'revenue')
    fig_top_rev = px.bar(
        top_rev, x='item_id', y='revenue',
        title="Top 10 Items by Revenue"
    )

    top_profit = df_curr.groupby('item_id', as_index=False)['profit'].sum().nlargest(10,'profit')
    fig_top_profit = px.bar(
        top_profit, x='item_id', y='profit',
        title="Top 10 Items by Profit"
    )

    return fig_rev_diff, fig_profit_diff, fig_lift, fig_top_rev, fig_top_profit

if __name__ == '__main__':
    app.run(debug=True)

# unused

In [ ]:

def create_item_selector(df):
    """
    Create an interactive widget for selecting and visualizing items
    """
    # Get available items
    available_items = get_available_items(df)
    
    # Create dropdown options (item_id: name format)
    dropdown_options = [(f"{row['item_id']}: {row['name']}", row['item_id']) 
                       for _, row in available_items.iterrows()]
    
    # Create widgets
    item_dropdown = widgets.Dropdown(
        options=dropdown_options,
        description='Select Item:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='600px')
    )
    
    # Multi-select for comparing multiple items
    multi_select = widgets.SelectMultiple(
        options=dropdown_options,
        description='Compare Items:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='600px', height='150px')
    )
    
    # Buttons
    plot_single_btn = widgets.Button(description="Plot Single Item", button_style='primary')
    plot_compare_btn = widgets.Button(description="Compare Multiple Items", button_style='success')
    show_stats_btn = widgets.Button(description="Show Statistics", button_style='info')
    
    # Output area
    output = widgets.Output()
    
    def plot_single_item(b):
        with output:
            clear_output(wait=True)
            if item_dropdown.value:
                plot_price_variance(df, item_dropdown.value)
    
    def plot_multiple_items(b):
        with output:
            clear_output(wait=True)
            if multi_select.value:
                plot_multiple_price_variance(df, list(multi_select.value))
    
    def show_item_statistics(b):
        with output:
            clear_output(wait=True)
            if multi_select.value:
                show_price_statistics(df, list(multi_select.value))
            elif item_dropdown.value:
                show_price_statistics(df, [item_dropdown.value])
    
    # Bind button clicks
    plot_single_btn.on_click(plot_single_item)
    plot_compare_btn.on_click(plot_multiple_items)
    show_stats_btn.on_click(show_item_statistics)
    
    # Layout
    ui = widgets.VBox([
        widgets.HTML("<h3>Item Price Analysis Dashboard</h3>"),
        widgets.HTML(f"<p>Total items available: {len(available_items)}</p>"),
        widgets.HTML("<h4>Single Item Analysis:</h4>"),
        item_dropdown,
        plot_single_btn,
        widgets.HTML("<h4>Multi-Item Comparison:</h4>"),
        multi_select,
        widgets.HBox([plot_compare_btn, show_stats_btn]),
        output
    ])
    
    return ui

def plot_multiple_price_variance(df, item_ids, max_items=5):
    """
    Plot price variance for multiple items on the same chart
    """
    if len(item_ids) > max_items:
        print(f"Too many items selected. Showing first {max_items} items.")
        item_ids = item_ids[:max_items]
    
    plt.figure(figsize=(15, 8))
    
    colors = plt.cm.Set3(np.linspace(0, 1, len(item_ids)))
    
    for i, item_id in enumerate(item_ids):
        # Filter data for this item
        d = df[df['item_id'] == item_id].sort_values('day').copy()
        if d.empty:
            continue
            
        # Convert price columns to numeric
        price_cols = ['regular_price', 'sale_price', 'selling_price']
        for col in price_cols:
            if col in d.columns:
                d[col] = pd.to_numeric(d[col], errors='coerce')
        
        item_name = d['name'].iloc[0]
        color = colors[i]
        
        # Plot selling price (most important for comparison)
        if 'selling_price' in d.columns:
            selling_data = d[d['selling_price'].notna()]
            if not selling_data.empty:
                plt.plot(selling_data['day'], selling_data['selling_price'], 
                        label=f'{item_id}: {item_name[:30]}...', 
                        marker='o', linewidth=2, color=color)
    
    plt.xlabel('Date')
    plt.ylabel('Selling Price ($)')
    plt.title('Price Comparison Across Multiple Items')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:.2f}'))
    plt.tight_layout()
    plt.show()